<a href="https://colab.research.google.com/github/railman-misaka/twikit2/blob/main/twikit_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twikit -q

## アカウント認証

In [ ]:
import asyncio
from twikit import Client

# 日本語なら"ja"
client = Client("ja")

USERNAME = "**************"
PASSWORD = "**************"
# EMAIL = "**************"

# EMAILは不要と思われますがエラーの場合はセットしてみて下さい
await client.login(
    auth_info_1=USERNAME,
    # auth_info_2=EMAIL,
    password=PASSWORD
)

client.save_cookies("cookies.json")

## Cookieファイルで認証

In [ ]:
import asyncio
from twikit import Client

client = Client()

client.load_cookies('cookies.json')

## 画像と動画ダウンロード
**※ユーザ名（@～のやつ）と投稿期間を指定できるようにしましたが直接キーワードを編集して大丈夫です**

In [ ]:
import re
import requests
import os
from datetime import datetime,timezone

pt = r"(.*)\?.*"

# 最大取得数（目安ですのでこれを超えることがあります）
MAX_COUNT = 100   #@param {type:"number"}

# ユーザ名と投稿期間（from～to）を指定
user_name = "sayaka_okada219" #@param {type:"string"}
date_from = "2024-07-01"  #@param {type:"string"}
date_until = "2024-08-01" #@param {type:"string"}
# @markdown **※MAX_COUNTは目安です。date_untilの日の投稿は含まれません。**

# 直接キーワードを指定しても構いません
keyword = f"from:{user_name} since:{date_from} until:{date_until}"

# キーワードからポスト取得
tweets = await client.search_tweet(query=keyword, product="Latest")

# フォルダ作成
fol_name = user_name + "_" + date_from
os.makedirs(fol_name, exist_ok=True)

count = 0

while count < MAX_COUNT:

    for tweet in tweets:

        if not tweet.media:
            continue

        for media in tweet.media:

            media.get("video_info")   # videoかどうか判定

            if media.get("video_info"):

                media_url = media["video_info"]["variants"][-1]["url"]
                result = re.search(pt, media_url)

                if result:
                    media_url = result.group(1)   # ?tag=～を削除

            else:
                # videoがなければ画像を取得
                media_url = media.get("media_url_https")

            response = requests.get(media_url)
            file_name = media_url[media_url.rfind('/') + 1:]

            with open(fol_name + "/" +file_name, 'wb') as f:
                f.write(response.content)

            count += 1

    print(str(count) + "件ダウンロード")

    tweets = await tweets.next()

    if not tweets:
        break

10件ダウンロード


# 対象フォルダのZIP化

In [ ]:
!zip -r $fol_name $fol_name